In [1]:
#Importing the necessary libraries
import pandas as pd
import minsearch
import requests 

#LLM Libraries
#!pip install mistralAI
import os
from mistralai import Mistral


In [2]:
# geting the data
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
#Using miniserach data retrival and indexing the data
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)


#Data retrival  
#query = "Can I join the course if it has already started?"
#for result in results:
#    print(result)

In [4]:
# LLM Key and model configuration
api_key='PkfJgpnzbOwxqJEweXH3NYreNrlRnBG9'
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [5]:
def search(query):
    filter_dict = {"course": "data-engineering-zoomcamp"}
    boost_dict = {"question": 3, "text": 1, "section": 1}

    results = index.search(
        query =query, 
        filter_dict =filter_dict,
        boost_dict =boost_dict,
        num_results=5
    )

    return results

In [10]:
def build_prompt(query,search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = " "
    for doc in search_results:
        context = context + f"section : {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query,context=context).strip()
    return prompt

In [7]:
def llm(prompt):
    chat_response = client.chat.complete(
    model = model,
    messages = [
        {
            "role": "user",
            "content": prompt,
        },
    ]
)
   
    return chat_response.choices[0].message.content

In [8]:
def rag(query):
  search_results = search(query)
  prompt =build_prompt(query,search_results)
  answer =llm(prompt)
  return answer

In [11]:
rag('how to enroll the cours')

'To enroll in the course, please follow the instructions provided in the course registration guide or contact the course administrator for further assistance.'